# Workflows and Agents
- Doc: https://langchain-ai.github.io/langgraph/tutorials/workflows/

In [ ]:
from dotenv import load_dotenv

# API-KEY 읽어오기
load_dotenv()

In [ ]:
from langchain.chat_models import init_chat_model

# 모델 초기화
llm = init_chat_model("google_genai:gemini-2.5-flash")

## Building Blocks: The Augmented LLM
- Doc: https://langchain-ai.github.io/langgraph/tutorials/workflows/#building-blocks-the-augmented-llm

In [ ]:
from pydantic import BaseModel, Field

# 에이전트의 상태 정의 클래스
class SearchQuery(BaseModel):
    # 웹 검색 쿼리
    search_query: str = Field(None, description="Query that is optimized web search.")
    # 사용자의 요청과 관련된 이유
    justification: str = Field(
        None, description="Why this query is relevant to the user's request."
    )


# 상태 기반 워크플로우 생성
structured_llm = llm.with_structured_output(SearchQuery)

# LLM을 호출하여 사용자 입력에 대한 응답 생성.
output = structured_llm.invoke("수학 성적이 좋은 학생은 과학 성적도 좋을 수 있어?")

# 결과 확인
print(output)

In [ ]:
# 간단한 곱셈 도구(multiply)
def multiply(a: int, b: int) -> int:
    return a * b

# LLM이 도구 호출 여부 판단
llm_with_tools = llm.bind_tools([multiply])

# 사용자 입력: 곱셈 요청("2 x 3 = ?")
msg = llm_with_tools.invoke("각각 8개씩 들어있는 사과 상자가 7개 있어, 모두 몇 개의 사과가 있는지 알려줘.")

# 도구 호출 정보 확인
print(f"도구 호출 정보: {msg.tool_calls}")

In [ ]:
# 도구 실행
if msg.tool_calls[0]['name'] == 'multiply':
    result = multiply(**msg.tool_calls[0]['args'])
    print(f"도구 실행 결과: {result}")

## Prompt chaining
- Doc: https://langchain-ai.github.io/langgraph/tutorials/workflows/#prompt-chaining

In [ ]:
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from IPython.display import Image, display


# 에이전트의 상태 정의 클래스
class State(TypedDict):
    topic: str           # 주제 (사용자 입력)
    joke: str            # 초기 농담
    improved_joke: str   # 개선된 농담
    final_joke: str      # 최종적으로 다듬어진 농담


# 첫 번째 LLM 호출: 최초 농담 생성 함수
def generate_joke(state: State):
    """First LLM call to generate initial joke"""
    # 주제(topic)를 기반으로 LLM에게 농담 생성 요청
    msg = llm.invoke(f"Write a short joke about {state['topic']}")
     # 생성된 농담을 상태에 저장
    return {"joke": msg.content}


# 농담에 펀치라인이 있는지 확인하는 게이트 함수 (펀치라인: 농담에서 가장 재미있고 결정적인 부분)
def check_punchline(state: State):
    """Gate function to check if the joke has a punchline"""

    # 농담에 "?" 또는 "!"가 포함되어 있으면 펀치라인이 있다고 간주
    if "?" in state["joke"] or "!" in state["joke"]:
        return "Pass"  # 펀치라인이 있으면 통과
    return "Fail"      # 펀치라인이 없으면 실패


# 두 번째 LLM 호출: 농담를 더 재미있게 개선하는 함수
def improve_joke(state: State):
    """Second LLM call to improve the joke"""

    # 기존 농담에 말장난(wordplay)을 추가하여 개선 요청
    msg = llm.invoke(f"Make this joke funnier by adding wordplay: {state['joke']}")
    # 개선된 농담를 상태에 저장
    return {"improved_joke": msg.content}


# 세 번째 LLM 호출: 농담에 반전을 추가하는 함수
def polish_joke(state: State):
    """Third LLM call for final polish"""

    # 개선된 농담에 놀라운 반전(twist)을 추가 요청
    msg = llm.invoke(f"Add a surprising twist to this joke: {state['improved_joke']}")
    # 최종 다듬어진 농담를 상태에 저장
    return {"final_joke": msg.content}


# 상태 기반 워크플로우 생성
workflow = StateGraph(State)

# 노드를 워크플로우에 추가
workflow.add_node("generate_joke", generate_joke)
workflow.add_node("improve_joke", improve_joke)
workflow.add_node("polish_joke", polish_joke)

# 시작 지점에서 초기 농담 생성으로 이동
workflow.add_edge(START, "generate_joke")  
# 펀치라인 확인 {실패: 종료, 통과: 개선}
workflow.add_conditional_edges(
    "generate_joke",
    check_punchline,
    {"Fail": END, "Pass": "improve_joke"}
)
# 개선 후 반전 추가 단계로 이동
workflow.add_edge("improve_joke", "polish_joke") 
# 다듬기 후 워크플로우 종료
workflow.add_edge("polish_joke", END)

# 워크플로우 컴파일, 실행 가능한 워크플로우 생성.
chain = workflow.compile()

# 워크플로우 그래프를 이미지로 시각화하여 출력
display(Image(chain.get_graph().draw_mermaid_png()))

In [ ]:
# 워크플로우 실행 및 결과 출력
state = chain.invoke({"topic": "장화 신은 고양이"})
print("Initial joke:")
print(state["joke"])
print("\n", "=" * 80,"\n")
if "improved_joke" in state:
    print("Improved joke:")
    print(state["improved_joke"])
    print("\n", "=" * 80,"\n")

    print("Final joke:")
    print(state["final_joke"])
else:
    print("Joke failed quality gate - no punchline detected!")

## Parallelization
- Doc: https://langchain-ai.github.io/langgraph/tutorials/workflows/#parallelization

In [ ]:
# 에이전트의 상태 정의 클래스
class State(TypedDict):
    topic: str            # 생성할 콘텐츠의 주제
    joke: str             # 주제에 기반한 농담
    story: str            # 주제에 기반한 이야기
    poem: str             # 주제에 기반한 시
    combined_output: str  # 농담, 이야기, 시를 결합한 최종 출력


# 첫 번째 LLM 호출: 주제에 맞는 농담을 생성하는 함수
def call_llm_1(state: State):
    """First LLM call to generate initial joke"""

    # 주제(topic)를 기반으로 LLM에게 농담 생성 요청
    msg = llm.invoke(f"Write a joke about {state['topic']}")
    # 생성된 농담을 상태에 저장
    return {"joke": msg.content}


# 두 번째 LLM 호출: 주제에 맞는 이야기를 생성하는 함수
def call_llm_2(state: State):
    """Second LLM call to generate story"""

    # 주제(topic)를 기반으로 LLM에게 이야기 생성 요청
    msg = llm.invoke(f"Write a story about {state['topic']}")
    # 생성된 이야기를 상태에 저장
    return {"story": msg.content}


# 세 번째 LLM 호출: 주제에 맞는 시를 생성하는 함수
def call_llm_3(state: State):
    """Third LLM call to generate poem"""

    # 주제(topic)를 기반으로 LLM에게 시 생성 요청
    msg = llm.invoke(f"Write a poem about {state['topic']}")
    # 생성된 시를 상태에 저장
    return {"poem": msg.content}


# 농담, 이야기, 시를 하나의 출력으로 모으는 함수
def aggregator(state: State):
    """Combine the joke and story into a single output"""

    # 주제를 포함한 헤더와 함께 농담, 이야기, 시를 결합
    combined = f"Here's a story, joke, and poem about {state['topic']}!\n\n"
    combined += f"STORY:\n{state['story']}\n\n"
    combined += f"JOKE:\n{state['joke']}\n\n"
    combined += f"POEM:\n{state['poem']}"
    # 결합된 결과를 상태에 저장
    return {"combined_output": combined}


# 상태 기반 워크플로우 생성
parallel_builder = StateGraph(State)

# 노드를 워크플로우에 추가
parallel_builder.add_node("call_llm_1", call_llm_1)  # 농담 생성 노드
parallel_builder.add_node("call_llm_2", call_llm_2)  # 이야기 생성 노드
parallel_builder.add_node("call_llm_3", call_llm_3)  # 시 생성 노드
parallel_builder.add_node("aggregator", aggregator)  # 결과 결합 노드

# 노드 간 엣지(연결)를 추가하여 병렬 처리 구조 정의
parallel_builder.add_edge(START, "call_llm_1")  # 시작 지점에서 농담 생성 노드로 이동
parallel_builder.add_edge(START, "call_llm_2")  # 시작 지점에서 이야기 생성 노드로 이동
parallel_builder.add_edge(START, "call_llm_3")  # 시작 지점에서 시 생성 노드로 이동
parallel_builder.add_edge("call_llm_1", "aggregator")  # 농담 생성 후 aggregator로 이동
parallel_builder.add_edge("call_llm_2", "aggregator")  # 이야기 생성 후 aggregator로 이동
parallel_builder.add_edge("call_llm_3", "aggregator")  # 시 생성 후 aggregator로 이동
parallel_builder.add_edge("aggregator", END)           # aggregator에서 워크플로우 종료

# 워크플로우 컴파일
parallel_workflow = parallel_builder.compile()

# 워크플로우 그래프를 이미지로 시각화하여 출력
display(Image(parallel_workflow.get_graph().draw_mermaid_png()))

In [ ]:
# 워크플로우 실행
events = parallel_workflow.stream({"topic": "호랑이 담배"})
# 이벤트 출력
final_event = None
for event in events:
    print("=" * 80)
    print(event)
    final_event = event  # 마지막 이벤트 저장

In [ ]:
# 최종 결과 출력
print(final_event["aggregator"]["combined_output"])

## Routing
- Doc: https://langchain-ai.github.io/langgraph/tutorials/workflows/#routing

In [ ]:
from typing_extensions import Literal
from langchain_core.messages import HumanMessage, SystemMessage


# 라우팅위 위한 구조화된 출력 스키마 정의 클래스
class Route(BaseModel):
    # 다음 단계로 라우팅할 경로
    step: Literal["poem", "story", "joke"] = Field(
        None, description="The next step in the routing process"
    )

# LLM이 구조화된 출력(Route 스키마)을 하도록
router = llm.with_structured_output(Route)

# 에이전트의 상태 정의 클래스
class State(TypedDict):
    input: str     # 사용자의 입력 요청
    decision: str  # 라우터가 결정한 다음 단계 (poem, story, joke)
    output: str    # 최종 생성된 출력 (농담, 이야기, 시)


# 주제에 맞는 이야기를 생성하는 함수
def llm_call_story(state: State):
    """Write a story"""
    # 입력을 기반으로 LLM에게 이야기 생성 요청
    result = llm.invoke(state["input"])
    # 생성된 이야기를 상태의 output에 저장
    return {"output": result.content}

# 주제에 맞는 농담을 생성하는 함수
def llm_call_joke(state: State):
    """Write a joke"""
    # 입력을 기반으로 LLM에게 농담 생성 요청
    result = llm.invoke(state["input"])
    # 생성된 농담을 상태의 output에 저장
    return {"output": result.content}

# 주제에 맞는 시를 생성하는 함수
def llm_call_poem(state: State):
    """Write a poem"""
    # 입력을 기반으로 LLM에게 시 생성 요청
    result = llm.invoke(state["input"])
    # 생성된 시를 상태의 output에 저장
    return {"output": result.content}

# 입력을 적절한 노드(story, joke, poem)로 라우팅하는 함수
def llm_call_router(state: State):
    """Route the input to the appropriate node"""

    # LLM에게 입력을 분석하고 라우팅 결정을 요청 (구조화된 출력)
    decision = router.invoke(
        [
            SystemMessage(
                content="Route the input to story, joke, or poem based on the user's request."
            ),
            HumanMessage(content=state["input"]),
        ]
    )

    return {"decision": decision.step}


# 라우팅 결정을 기반으로 다음 노드를 선택
def route_decision(state: State):
    if state["decision"] == "story":
        return "llm_call_story"  # 이야기 생성 노드로 이동
    elif state["decision"] == "joke":
        return "llm_call_joke"  # 농담 생성 노드로 이동
    elif state["decision"] == "poem":
        return "llm_call_poem"  # 시 생성 노드로 이동


# 상태 기반 워크플로우 생성
router_builder = StateGraph(State)

# 노드를 워크플로우에 추가
router_builder.add_node("llm_call_story", llm_call_story)            # 이야기 생성 노드
router_builder.add_node("llm_call_joke", llm_call_joke)            # 농담 생성 노드
router_builder.add_node("llm_call_poem", llm_call_poem)            # 시 생성 노드
router_builder.add_node("llm_call_router", llm_call_router)  # 라우팅 결정 노드

# 노드 간 엣지(연결)를 추가하여 워크플로우 구조 정의
router_builder.add_edge(START, "llm_call_router")  # 시작 지점에서 라우팅 노드로 이동
router_builder.add_conditional_edges(              # route_decision 결과에 따라 다음 노드 선택
    "llm_call_router",
    route_decision,
    {
        "llm_call_story": "llm_call_story",
        "llm_call_joke": "llm_call_joke",
        "llm_call_poem": "llm_call_poem",
    },
)
router_builder.add_edge("llm_call_story", END)  # 이야기 생성 후 워크플로우 종료
router_builder.add_edge("llm_call_joke", END)  # 농담 생성 후 워크플로우 종료
router_builder.add_edge("llm_call_poem", END)  # 시 생성 후 워크플로우 종료

# 워크플로우 컴파일
router_workflow = router_builder.compile()

# 워크플로우 그래프를 이미지로 시각화하여 출력
display(Image(router_workflow.get_graph().draw_mermaid_png()))

In [ ]:
# 워크플로우 실행
events = router_workflow.stream({"input": "고양이 관련한 농담을 만들어줘"})
# 이벤트 출력
final_event = None
for event in events:
    print("=" * 80)
    print(event)
    final_event = event  # 마지막 이벤트 저장

In [ ]:
# 최종 결과 출력
print(final_event["llm_call_joke"]["output"])

In [ ]:
# 워크플로우 실행
events = router_workflow.stream({"input": "고양이 관련한 이야기를 만들어줘"})
# 이벤트 출력
final_event = None
for event in events:
    print("=" * 80)
    print(event)
    final_event = event  # 마지막 이벤트 저장

In [ ]:
# 최종 결과 출력
print(final_event["llm_call_story"]["output"])

In [ ]:
# 워크플로우 실행
events = router_workflow.stream({"input": "고양이 관련한 시를 만들어줘"})
# 이벤트 출력
final_event = None
for event in events:
    print("=" * 80)
    print(event)
    final_event = event  # 마지막 이벤트 저장

In [ ]:
# 최종 결과 출력
print(final_event["llm_call_poem"]["output"])

## Orchestrator-Worker
- Doc: https://langchain-ai.github.io/langgraph/tutorials/workflows/#orchestrator-worker

In [ ]:
from typing import Annotated, List


# 보고서 섹션의 구조를 정의하는 클래스
class Section(BaseModel):
    name: str = Field(         # 보고서 섹션의 제목
        description="Name for this section of the report.",
    )
    description: str = Field(  # 섹션에서 다룰 주요 주제와 개념의 개요
        description="Brief overview of the main topics and concepts to be covered in this section.",
    )


# 여러 섹션을 포함하는 보고서 구조를 정의하는 클래스
class Sections(BaseModel):
    sections: List[Section] = Field(  # 보고서의 섹션 목록
        description="Sections of the report.",
    )


# LLM이 구조화된 출력(Sections 스키마)을 하도록
planner = llm.with_structured_output(Sections)

In [ ]:
import operator
from langgraph.types import Send


# 에이전트의 상태 정의 클래스
class State(TypedDict):
    topic: str                      # 보고서의 주제
    sections: list[Section]         # 보고서의 섹션 목록
    completed_sections: Annotated[  # 완성된 섹션 목록 (operator.add로 추가)
        list, operator.add
    ]
    final_report: str               # 최종 보고서 내용


# 개별 섹션 생성 워커의 상태 정의 클래스
class WorkerState(TypedDict):
    section: Section                # 처리할 개별 섹션


# 보고서 계획을 생성하는 오케스트레이터 함수
def orchestrator(state: State):
    """Orchestrator that generates a plan for the report"""

    # Sections 스키마 생성 요청
    report_sections = planner.invoke(
        [
            # 보고서 계획 생성 요청
            SystemMessage(content="Generate a plan for the report."),
            # 주제 제공
            HumanMessage(content=f"Here is the report topic: {state['topic']}"),
        ]
    )
    # 생성된 섹션 계획을 상태에 저장
    return {"sections": report_sections.sections}


# 보고서의 개별 섹션을 작성하는 워커 함수
def llm_call(state: WorkerState):
    """Worker writes a section of the report"""

    # Generate section
    # LLM을 통해 섹션 이름과 설명을 기반으로 섹션 내용 생성
    section = llm.invoke(
        [
            SystemMessage(
                content="Write a report section following the provided name and description."
                        " Include no preamble for each section."
                        " Use markdown formatting."
                # 섹션 작성 지침: 서문 없이 마크다운 형식으로 작성
            ),
            HumanMessage(
                content=f"Here is the section name: {state['section'].name} and description: {state['section'].description}"
                # 섹션 작성 지침: 서문 없이 마크다운 형식으로 작성
            ),
        ]
    )

    # 작성된 섹션을 completed_sections에 저장
    return {"completed_sections": [section.content]}


# 모든 섹션을 합쳐 최종 보고서를 생성하는 함수
def synthesizer(state: State):
    """Synthesize full report from sections"""

    # 완성된 섹션 리스트 가져오기
    completed_sections = state["completed_sections"]

    # 섹션들을 연결하여 문자열로 포맷
    completed_report_sections = "\n\n---\n\n".join(completed_sections)

    # 최종 보고서를 상태에 저장
    return {"final_report": completed_report_sections}


# 조건부 엣지 함수: 보고서 계획의 각 섹션에 워커를 할당하는 함수
def assign_workers(state: State):
    """Assign a worker to each section in the plan"""

    # 섹션마다 병렬로 llm_call 워커를 실행하도록 Send API 사용
    return [Send("llm_call", {"section": s}) for s in state["sections"]]


# 워크플로우 그래프 구축
orchestrator_worker_builder = StateGraph(State)

# 노드를 워크플로우에 추가
orchestrator_worker_builder.add_node("orchestrator", orchestrator)  # 보고서 계획 생성 노드
orchestrator_worker_builder.add_node("llm_call", llm_call)          # 섹션 작성 노드
orchestrator_worker_builder.add_node("synthesizer", synthesizer)    # 최종 보고서 통합 노드

# 노드 간 엣지(연결)를 추가하여 워크플로우 구조 정의
orchestrator_worker_builder.add_edge(START, "orchestrator")      # 시작 지점에서 오케스트레이터로 이동
orchestrator_worker_builder.add_conditional_edges(               # 오케스트레이터에서 섹션별로 병렬 이동
    "orchestrator", assign_workers, ["llm_call"]
)
orchestrator_worker_builder.add_edge("llm_call", "synthesizer")  # 섹션 작성 후 합성 노드로 이동
orchestrator_worker_builder.add_edge("synthesizer", END)         # 합성 후 워크플로우 종료

# 워크플로우 컴파일
orchestrator_worker = orchestrator_worker_builder.compile()

# 워크플로우 그래프를 이미지로 시각화하여 출력
display(Image(orchestrator_worker.get_graph().draw_mermaid_png()))

In [ ]:
# 워크플로우 실행
events = orchestrator_worker.stream({"topic": "LLM scaling laws에 관련한 보고서를 작성해줘"})
# 이벤트 출력
final_event = None
for event in events:
    print("=" * 80)
    print(event)
    final_event = event  # 마지막 이벤트 저장

In [ ]:
# 최종 보고서를 마크다운 형식으로 출력
from IPython.display import Markdown
Markdown(final_event["synthesizer"]["final_report"])

## Evaluator-optimizer
- Doc: https://langchain-ai.github.io/langgraph/tutorials/workflows/#evaluator-optimizer

In [ ]:
# 에이전트의 상태 정의 클래스
class State(TypedDict):
    topic: str        # 농담의 주제
    joke: str         # 생성된 농담
    feedback: str     # 농담 개선을 위한 피드백
    funny_or_not: str # 농담이 재미있는지 여부 ("funny" 또는 "not funny")


# 평가를 위한 구조화된 출력 스키마 정의
class Feedback(BaseModel):
    grade: Literal["funny", "not funny"] = Field(  # 농담의 재미 여부 판단
        description="Decide if the joke is funny or not.",
    )
    feedback: str = Field(                         # 재미없을 경우 개선 피드백
        description="If the joke is not funny, provide feedback on how to improve it.",
    )


# LLM이 구조화된 출력(Feedback 스키마)을 하도록
evaluator = llm.with_structured_output(Feedback)


# LLM이 주제에 맞는 농담을 생성하는 함수
def llm_call_generator(state: State):
    """LLM generates a joke"""

    if state.get("feedback"):  # 피드백이 있는 경우, 피드백을 반영하여 농담 생성
        msg = llm.invoke(
            f"Write a joke about {state['topic']} but take into account the feedback: {state['feedback']}"
        )
    else:                      # 피드백이 없는 경우, 주제만으로 농담 생성
        msg = llm.invoke(f"Write a joke about {state['topic']}")
    # 생성된 농담을 상태에 저장
    return {"joke": msg.content}

# LLM이 농담을 평가하는 함수
def llm_call_evaluator(state: State):
    """LLM evaluates the joke"""

    # 농담을 평가하여 재미 여부와 피드백 생성
    grade = evaluator.invoke(f"Grade the joke {state['joke']}")
    # 평가 결과(재미 여부)와 피드백을 상태에 저장
    return {"funny_or_not": grade.grade, "feedback": grade.feedback}


# 조건부 엣지 함수: 평가 결과에 따라 농담 생성기로 돌아가거나 종료하는 함수
def route_joke(state: State):
    """Route back to joke generator or end based upon feedback from the evaluator"""
    if state["funny_or_not"] == "funny":        # 농담이 재미있다면 워크플로우 종료
        return "Accepted"
    elif state["funny_or_not"] == "not funny":  # 농담이 재미없다면 피드백과 함께 생성기로 돌아감
        return "Rejected + Feedback"


# 워크플로우 그래프 구축
optimizer_builder = StateGraph(State)

# 노드를 워크플로우에 추가
optimizer_builder.add_node("llm_call_generator", llm_call_generator)  # 농담 생성 노드
optimizer_builder.add_node("llm_call_evaluator", llm_call_evaluator)  # 농 KY 평가 노드

# 노드 간 엣지(연결)를 추가하여 워크플로우 구조 정의
optimizer_builder.add_edge(START, "llm_call_generator")  # 시작 지점에서 농담 생성 노드로 이동
optimizer_builder.add_edge("llm_call_generator", "llm_call_evaluator")  # 생성 후 평가 노드로 이동
optimizer_builder.add_conditional_edges(
    "llm_call_evaluator",
    route_joke,  # route_joke 함수가 반환한 값에 따라 다음 단계 결정
    {
        "Accepted": END,                              # 재미있으면 종료
        "Rejected + Feedback": "llm_call_generator",  # 재미없으면 피드백과 함께 생성기로 돌아감
    },
)

# 워크플로우 컴파일
optimizer_workflow = optimizer_builder.compile()

# 워크플로우 그래프를 이미지로 시각화하여 표시
display(Image(optimizer_workflow.get_graph().draw_mermaid_png()))

In [ ]:
# 워크플로우 실행
events = optimizer_workflow.stream({"topic": "고양이"})
# 이벤트 출력
final_event = None
for event in events:
    print("=" * 80)
    print(event)
    if "llm_call_generator" in event:
        final_event = event  # llm_call_generator의 마지막 이벤트 저장

In [ ]:
# 최종 결과 출력
print(final_event["llm_call_generator"]["joke"])

# Agent
- Doc: https://langchain-ai.github.io/langgraph/tutorials/workflows/#agent

In [ ]:
from langchain_core.tools import tool


# 두 정수의 곱을 계산 하는 도구
@tool
def multiply(a: int, b: int) -> int:
    """Multiply a and b.

    Args:
        a: first int
        b: second int
    """
    return a * b


# 두 정수의 합을 계산 하는 도구
@tool
def add(a: int, b: int) -> int:
    """Adds a and b.

    Args:
        a: first int
        b: second int
    """
    return a + b


# 두 정수의 나누기를 계산 하는 도구
@tool
def divide(a: int, b: int) -> float:
    """Divide a and b.

    Args:
        a: first int
        b: second int
    """
    return a / b


# 사용 가능한 도구 목록
tools = [add, multiply, divide]
# 도구 이름을 키로 한 딕셔너리 생성
tools_by_name = {tool.name: tool for tool in tools}
# LLM이 도구 호출 여부 판단
llm_with_tools = llm.bind_tools(tools)

In [ ]:
from langgraph.graph import MessagesState, START, END
from langchain_core.messages import SystemMessage, HumanMessage, ToolMessage


# LLM이 도구 호출 여부를 결정하는 함수
def llm_call(state: MessagesState):
    """LLM decides whether to call a tool or not"""

    return {
        "messages": [
            llm_with_tools.invoke(
                [
                    SystemMessage(
                        content="You are a helpful assistant tasked with performing arithmetic on a set of inputs."
                    )
                ]
                + state["messages"]
            )
        ]
    }


# 도구 호출을 수행하는 함수
def tool_node(state: dict):
    """Performs the tool call"""

    result = []
    for tool_call in state["messages"][-1].tool_calls:
        tool = tools_by_name[tool_call["name"]]       # 호출된 도구 이름으로 도구 선택
        observation = tool.invoke(tool_call["args"])  # 도구 실행
        # 도구 호출 결과로 ToolMessage 생성
        result.append(ToolMessage(content=observation, tool_call_id=tool_call["id"]))
    return {"messages": result}  # 도구 호출 결과를 메시지로 반환


# 조건부 엣지 함수: LLM이 도구를 호출했는지 여부에 따라 다음 단계를 결정하는 함수
def should_continue(state: MessagesState) -> Literal["environment", END]:
    """Decide if we should continue the loop or stop based upon whether the LLM made a tool call"""

    messages = state["messages"]
    last_message = messages[-1]
    if last_message.tool_calls:  # LLM이 도구를 호출했으면 tool_node 노드로 이동
        return "Action"
    return END                   # 도구 호출이 없으면 워크플로우 종료


# 워크플로우 그래프 구축
agent_builder = StateGraph(MessagesState)

# 노드를 워크플로우에 추가
agent_builder.add_node("llm_call", llm_call)      # LLM 호출 노드
agent_builder.add_node("environment", tool_node)  # 도구 실행 노드

# 노드 간 엣지(연결)를 추가하여 워크플로우 구조 정의
agent_builder.add_edge(START, "llm_call")         # 시작 지점에서 LLM 호출 노드로 이동
agent_builder.add_conditional_edges(
    "llm_call",
    should_continue, # should_continue 함수의 반환값에 따라 다음 노드 결정
    {
        "Action": "environment",  # 도구 호출 시 environment로 이동
        END: END,                 # 도구 호출 없으면 종료
    },
)
agent_builder.add_edge("environment", "llm_call")  # 도구 실행 후 다시 LLM 호출로 이동

# 워크플로우 컴파일
agent = agent_builder.compile()

# 워크플로우 그래프를 이미지로 시각화하여 출력
display(Image(agent.get_graph(xray=True).draw_mermaid_png()))

In [ ]:
# 워크플로우 실행
user_input = "사과 6개와 배 5개가 들어있는 상자가 5개일 때, 과일의 총 개수를 구하시오."
events = agent.stream({"messages": [{"role": "user", "content": user_input}]})
for event in events:
    for value in event.values():
        value["messages"][-1].pretty_print()